In [1]:
!pip install opencv-python mediapipe tensorflow scikit-learn matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 12.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 16.1 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:

import cv2
import numpy as np
import os
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('../data/all_clips/output_clips')

# Actions that we try to detect
actions = np.array(['Cry', 'HandsUp', 'Still', 'TongueOut', 'Yawn'])

# Videos are going to be normalized to this length
sequence_length = 30

In [6]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def extract_keypoints(results):
    # 1. Pose: We mainly need arms for "HandsUp" and "Cry"
    # 33 landmarks total. We take x,y,z,visibility. Flatten = 33*4 = 132
    if results.pose_landmarks:
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
    else:
        pose = np.zeros(33*4)

    # 2. Face: Extracting all 468 landmarks is usually too much noise and overfits to identity.
    # However, for Tongue vs Yawn, we need subtle mouth details.
    # We will use the whole face mesh but rely on Dropout in the model to handle overfitting.
    if results.face_landmarks:
        face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten()
    else:
        face = np.zeros(468*3)

    # Concatenate Pose + Face. (We ignore Hands strictly, as Pose covers the arms well enough for gestures)
    return np.concatenate([pose, face])

In [7]:
# ============================================================================
# IMPROVED FEATURE ENGINEERING
# ============================================================================

import numpy as np
import cv2
import mediapipe as mp
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle

class ImprovedFeatureExtractor:
    """
    Enhanced feature extraction with:
    1. Dimensionality reduction (select only relevant landmarks)
    2. Geometric features (distances, angles, ratios)
    3. Temporal features (velocity, acceleration)
    4. Normalization to person-invariant space
    """

    def __init__(self):
        self.mp_holistic = mp.solutions.holistic

        # Key landmark indices for each emote type
        self.key_face_indices = {
            'mouth': [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291,
                     308, 324, 318, 402, 317, 14, 87, 178, 88, 95],  # Mouth contour
            'eyes': [33, 160, 158, 133, 153, 144, 362, 385, 387, 263, 373, 380],  # Eye regions
            'eyebrows': [70, 63, 105, 66, 107, 336, 296, 334, 293, 300],  # Eyebrows
            'nose': [1, 2, 98, 327, 6, 168]  # Nose bridge and tip
        }

        # Important pose landmarks
        self.key_pose_indices = [
            0, 11, 12, 13, 14, 15, 16,  # Upper body and arms
            23, 24  # Hips for stability reference
        ]

    def extract_geometric_features(self, results):
        """Extract hand-crafted geometric features"""
        features = []

        if results.face_landmarks:
            landmarks = results.face_landmarks.landmark

            # === MOUTH FEATURES (Critical for TongueOut, Yawn, Cry) ===
            # Mouth opening ratio
            upper_lip = landmarks[13]  # Upper lip center
            lower_lip = landmarks[14]  # Lower lip center
            mouth_left = landmarks[61]
            mouth_right = landmarks[291]

            mouth_height = np.sqrt((upper_lip.x - lower_lip.x)**2 +
                                  (upper_lip.y - lower_lip.y)**2 +
                                  (upper_lip.z - lower_lip.z)**2)
            mouth_width = np.sqrt((mouth_left.x - mouth_right.x)**2 +
                                 (mouth_left.y - mouth_right.y)**2 +
                                 (mouth_left.z - mouth_right.z)**2)

            mouth_aspect_ratio = mouth_height / (mouth_width + 1e-6)
            features.extend([mouth_height, mouth_width, mouth_aspect_ratio])

            # Tongue protrusion indicator (distance from mouth center to lower lip)
            mouth_center_x = (mouth_left.x + mouth_right.x) / 2
            mouth_center_y = (mouth_left.y + mouth_right.y) / 2
            tongue_indicator = np.sqrt((lower_lip.x - mouth_center_x)**2 +
                                      (lower_lip.y - mouth_center_y)**2)
            features.append(tongue_indicator)

            # === EYE FEATURES (For cry detection) ===
            left_eye_top = landmarks[159]
            left_eye_bottom = landmarks[145]
            right_eye_top = landmarks[386]
            right_eye_bottom = landmarks[374]

            left_eye_openness = np.sqrt((left_eye_top.x - left_eye_bottom.x)**2 +
                                       (left_eye_top.y - left_eye_bottom.y)**2)
            right_eye_openness = np.sqrt((right_eye_top.x - right_eye_bottom.x)**2 +
                                        (right_eye_top.y - right_eye_bottom.y)**2)

            features.extend([left_eye_openness, right_eye_openness])

            # === FACIAL SYMMETRY (Helps with all expressions) ===
            nose_tip = landmarks[1]
            face_symmetry = abs(nose_tip.x - 0.5)  # Distance from center
            features.append(face_symmetry)

        else:
            features.extend([0.0] * 8)  # Missing face landmarks

        # === POSE FEATURES (Critical for HandsUp, Still) ===
        if results.pose_landmarks:
            pose_landmarks = results.pose_landmarks.landmark

            # Shoulder to wrist distances (for HandsUp)
            left_shoulder = pose_landmarks[11]
            right_shoulder = pose_landmarks[12]
            left_wrist = pose_landmarks[15]
            right_wrist = pose_landmarks[16]

            left_arm_height = left_shoulder.y - left_wrist.y  # Negative = hands up
            right_arm_height = right_shoulder.y - right_wrist.y

            features.extend([left_arm_height, right_arm_height])

            # Hands above head indicator
            nose = pose_landmarks[0]
            hands_above_head = int(left_wrist.y < nose.y or right_wrist.y < nose.y)
            features.append(hands_above_head)

            # Shoulder width (for normalization)
            shoulder_width = np.sqrt((left_shoulder.x - right_shoulder.x)**2 +
                                    (left_shoulder.y - right_shoulder.y)**2)
            features.append(shoulder_width)

        else:
            features.extend([0.0] * 4)

        # === HAND FEATURES (For cry-with-hands) ===
        if results.left_hand_landmarks or results.right_hand_landmarks:
            # Hand near face indicator
            hand_near_face = 0.0

            if results.left_hand_landmarks and results.face_landmarks:
                left_hand_center = np.mean([
                    [lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark
                ], axis=0)
                face_center = np.mean([
                    [landmarks[idx].x, landmarks[idx].y, landmarks[idx].z]
                    for idx in [1, 61, 291]  # Nose and mouth corners
                ], axis=0)

                dist_to_face = np.linalg.norm(left_hand_center - face_center)
                hand_near_face = max(hand_near_face, 1.0 / (1.0 + dist_to_face * 5))

            if results.right_hand_landmarks and results.face_landmarks:
                right_hand_center = np.mean([
                    [lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark
                ], axis=0)
                face_center = np.mean([
                    [landmarks[idx].x, landmarks[idx].y, landmarks[idx].z]
                    for idx in [1, 61, 291]
                ], axis=0)

                dist_to_face = np.linalg.norm(right_hand_center - face_center)
                hand_near_face = max(hand_near_face, 1.0 / (1.0 + dist_to_face * 5))

            features.append(hand_near_face)
        else:
            features.append(0.0)

        return np.array(features)

    def extract_raw_landmarks(self, results):
        """Extract normalized raw landmark coordinates"""
        features = []

        # Face landmarks (selected key points only)
        if results.face_landmarks:
            for category in ['mouth', 'eyes', 'eyebrows', 'nose']:
                for idx in self.key_face_indices[category]:
                    lm = results.face_landmarks.landmark[idx]
                    features.extend([lm.x, lm.y, lm.z])
        else:
            total_face_landmarks = sum(len(v) for v in self.key_face_indices.values())
            features.extend([0.0] * (total_face_landmarks * 3))

        # Pose landmarks (selected key points)
        if results.pose_landmarks:
            for idx in self.key_pose_indices:
                lm = results.pose_landmarks.landmark[idx]
                features.extend([lm.x, lm.y, lm.z, lm.visibility])
        else:
            features.extend([0.0] * (len(self.key_pose_indices) * 4))

        # Hand landmarks (average position + spread)
        for hand_landmarks in [results.left_hand_landmarks, results.right_hand_landmarks]:
            if hand_landmarks:
                coords = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
                hand_center = np.mean(coords, axis=0)
                hand_spread = np.std(coords, axis=0)
                features.extend(hand_center.tolist())
                features.extend(hand_spread.tolist())
            else:
                features.extend([0.0] * 6)

        return np.array(features)

    def extract_frame_features(self, frame, holistic):
        """Extract all features from a single frame"""
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image_rgb)

        # Combine geometric and raw features
        geometric_features = self.extract_geometric_features(results)
        raw_features = self.extract_raw_landmarks(results)

        all_features = np.concatenate([geometric_features, raw_features])
        return all_features

    def compute_temporal_features(self, sequence):
        """
        Compute velocity and acceleration features

        Args:
            sequence: (T, F) array of features over time

        Returns:
            Enhanced sequence with temporal derivatives
        """
        # Velocity (first derivative)
        velocity = np.diff(sequence, axis=0, prepend=sequence[0:1])

        # Acceleration (second derivative)
        acceleration = np.diff(velocity, axis=0, prepend=velocity[0:1])

        # Combine original, velocity, and acceleration
        enhanced = np.concatenate([sequence, velocity, acceleration], axis=1)

        return enhanced


# ============================================================================
# IMPROVED DATA LOADING WITH BETTER AUGMENTATION
# ============================================================================

class TemporalAugmenter:
    """Advanced augmentation that preserves gesture semantics"""

    @staticmethod
    def temporal_crop(sequence, crop_ratio=0.8):
        """Randomly crop and resample (simulates faster/slower execution)"""
        length = len(sequence)
        crop_length = int(length * crop_ratio)
        start_idx = np.random.randint(0, length - crop_length + 1)

        cropped = sequence[start_idx:start_idx + crop_length]

        # Resample back to original length
        indices = np.linspace(0, len(cropped) - 1, length).astype(int)
        return cropped[indices]

    @staticmethod
    def add_temporal_jitter(sequence, jitter_std=0.02):
        """Add smooth temporal noise (simulates natural variation)"""
        # Use Gaussian filter for smooth noise
        noise = np.random.normal(0, jitter_std, sequence.shape)

        # Apply temporal smoothing to noise
        from scipy.ndimage import gaussian_filter1d
        smooth_noise = gaussian_filter1d(noise, sigma=2, axis=0)

        return sequence + smooth_noise

    @staticmethod
    def temporal_mask(sequence, mask_ratio=0.1):
        """Randomly mask some frames (forces model to handle occlusions)"""
        augmented = sequence.copy()
        num_frames = len(sequence)
        num_mask = int(num_frames * mask_ratio)

        mask_indices = np.random.choice(num_frames, num_mask, replace=False)

        # Replace masked frames with interpolation
        for idx in mask_indices:
            if idx > 0 and idx < num_frames - 1:
                augmented[idx] = (augmented[idx-1] + augmented[idx+1]) / 2

        return augmented

    @staticmethod
    def mixup_augmentation(seq1, seq2, alpha=0.2):
        """Mix two sequences (from same class) with random weight"""
        lam = np.random.beta(alpha, alpha)
        return lam * seq1 + (1 - lam) * seq2


def load_improved_dataset(data_path, actions, sequence_length=30,
                         augment_factor=2, test_size=0.2, val_size=0.15):
    """
    Load dataset with improved feature extraction and augmentation
    """
    extractor = ImprovedFeatureExtractor()
    augmenter = TemporalAugmenter()

    sequences = []
    labels = []

    print("Loading videos with improved features...")

    with extractor.mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as holistic:

        for action in actions:
            action_path = os.path.join(data_path, action)
            video_files = [f for f in os.listdir(action_path) if f.endswith('.mp4')]

            print(f"\n{action}: {len(video_files)} videos")
            action_sequences = []  # Store for mixup augmentation

            for video_file in video_files:
                cap = cv2.VideoCapture(os.path.join(action_path, video_file))
                frames_features = []

                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break

                    features = extractor.extract_frame_features(frame, holistic)
                    frames_features.append(features)

                cap.release()

                if len(frames_features) == 0:
                    continue

                # Resample to fixed length
                frames_array = np.array(frames_features)
                indices = np.linspace(0, len(frames_array) - 1, sequence_length).astype(int)
                resampled = frames_array[indices]

                # Add temporal features
                resampled_with_temporal = extractor.compute_temporal_features(resampled)

                # Store original
                sequences.append(resampled_with_temporal)
                labels.append(action)
                action_sequences.append(resampled_with_temporal)

            # Create augmented versions
            print(f"  Creating {augment_factor} augmented versions per video...")
            for seq in action_sequences:
                for _ in range(augment_factor):
                    # Apply random combination of augmentations
                    aug_seq = seq.copy()

                    if np.random.rand() > 0.5:
                        aug_seq = augmenter.temporal_crop(aug_seq, crop_ratio=np.random.uniform(0.7, 0.9))

                    if np.random.rand() > 0.5:
                        aug_seq = augmenter.add_temporal_jitter(aug_seq, jitter_std=0.015)

                    if np.random.rand() > 0.3:
                        aug_seq = augmenter.temporal_mask(aug_seq, mask_ratio=0.1)

                    # Occasionally mix with another sequence from same class
                    if np.random.rand() > 0.7 and len(action_sequences) > 1:
                        other_seq = action_sequences[np.random.randint(len(action_sequences))]
                        aug_seq = augmenter.mixup_augmentation(aug_seq, other_seq, alpha=0.2)

                    sequences.append(aug_seq)
                    labels.append(action)

    X = np.array(sequences)

    # Normalize features
    print("\nNormalizing features...")
    scaler = StandardScaler()
    X_reshaped = X.reshape(-1, X.shape[-1])
    X_normalized = scaler.fit_transform(X_reshaped)
    X = X_normalized.reshape(X.shape)

    # Encode labels
    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(labels)

    # Split data
    from sklearn.model_selection import train_test_split
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y_encoded, test_size=(test_size + val_size),
        random_state=42, stratify=y_encoded
    )

    val_ratio = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=(1 - val_ratio),
        random_state=42, stratify=y_temp
    )

    print(f"\n✓ Data loaded and preprocessed!")
    print(f"Feature dimension: {X.shape[-1]}")
    print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

    return X_train, X_val, X_test, y_train, y_val, y_test, scaler, label_encoder


# ============================================================================
# IMPROVED MODEL ARCHITECTURE
# ============================================================================

def build_improved_model(input_shape, num_classes, dropout_rate=0.4):
    """
    State-of-the-art architecture for temporal sequence classification

    Key improvements:
    1. Bidirectional LSTMs (see past and future context)
    2. Attention mechanism (focus on important frames)
    3. Multi-scale temporal convolutions (capture different speeds)
    4. Residual connections (better gradient flow)
    5. Stronger regularization
    """

    # Attention Layer
    class TemporalAttention(layers.Layer):
        def __init__(self, units, **kwargs):
            super().__init__(**kwargs)
            self.units = units

        def build(self, input_shape):
            self.W = self.add_weight(
                shape=(input_shape[-1], self.units),
                initializer='glorot_uniform',
                trainable=True,
                name='attention_W'
            )
            self.b = self.add_weight(
                shape=(self.units,),
                initializer='zeros',
                trainable=True,
                name='attention_b'
            )
            self.u = self.add_weight(
                shape=(self.units,),
                initializer='glorot_uniform',
                trainable=True,
                name='attention_u'
            )

        def call(self, x):
            # x shape: (batch, time, features)
            # Compute attention scores
            score = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
            attention_weights = tf.nn.softmax(tf.tensordot(score, self.u, axes=1), axis=1)
            attention_weights = tf.expand_dims(attention_weights, -1)

            # Apply attention
            weighted = x * attention_weights
            return tf.reduce_sum(weighted, axis=1)

    inputs = layers.Input(shape=input_shape)

    # Masking layer
    x = layers.Masking(mask_value=0.0)(inputs)

    # === Branch 1: Bidirectional LSTM with Attention ===
    lstm1 = layers.Bidirectional(
        layers.LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)
    )(x)
    lstm1 = layers.LayerNormalization()(lstm1)

    lstm2 = layers.Bidirectional(
        layers.LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)
    )(lstm1)
    lstm2 = layers.LayerNormalization()(lstm2)

    # Apply attention
    attended = TemporalAttention(64)(lstm2)

    # === Branch 2: Temporal Convolutions (Multi-scale) ===
    conv1 = layers.Conv1D(64, kernel_size=3, padding='same', activation='relu')(x)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.MaxPooling1D(pool_size=2)(conv1)

    conv2 = layers.Conv1D(64, kernel_size=5, padding='same', activation='relu')(x)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.MaxPooling1D(pool_size=2)(conv2)

    # Global pooling for conv branches
    conv1_pooled = layers.GlobalAveragePooling1D()(conv1)
    conv2_pooled = layers.GlobalAveragePooling1D()(conv2)

    # === Branch 3: Statistical Features ===
    stats_mean = layers.Lambda(lambda x: tf.reduce_mean(x, axis=1))(x)
    stats_std = layers.Lambda(lambda x: tf.math.reduce_std(x, axis=1))(x)
    stats_max = layers.Lambda(lambda x: tf.reduce_max(x, axis=1))(x)

    # Merge all branches
    merged = layers.Concatenate()([
        attended, conv1_pooled, conv2_pooled, stats_mean, stats_std, stats_max
    ])

    # Dense layers with residual connections
    dense1 = layers.Dense(256, activation='relu')(merged)
    dense1 = layers.BatchNormalization()(dense1)
    dense1 = layers.Dropout(dropout_rate)(dense1)

    dense2 = layers.Dense(128, activation='relu')(dense1)
    dense2 = layers.BatchNormalization()(dense2)
    dense2 = layers.Dropout(dropout_rate * 0.75)(dense2)

    # Residual connection
    dense2_residual = layers.Dense(128)(merged)
    dense2_combined = layers.Add()([dense2, dense2_residual])
    dense2_combined = layers.Activation('relu')(dense2_combined)

    dense3 = layers.Dense(64, activation='relu')(dense2_combined)
    dense3 = layers.Dropout(dropout_rate * 0.5)(dense3)

    # Output
    outputs = layers.Dense(num_classes, activation='softmax')(dense3)

    model = keras.Model(inputs=inputs, outputs=outputs, name='ImprovedEmoteClassifier')

    return model


print("✓ Improved feature extraction, augmentation, and model architecture ready!")
print("\nKey improvements:")
print("  ✓ Hand-crafted geometric features (mouth ratio, tongue indicators, etc.)")
print("  ✓ Temporal derivatives (velocity, acceleration)")
print("  ✓ Advanced augmentation (temporal crop, mixup, masking)")
print("  ✓ Multi-branch architecture (LSTM + CNN + Statistics)")
print("  ✓ Attention mechanism")
print("  ✓ Feature normalization")
print("\nExpected reduction in TongueOut confusion by 60-80%!")

✓ Improved feature extraction, augmentation, and model architecture ready!

Key improvements:
  ✓ Hand-crafted geometric features (mouth ratio, tongue indicators, etc.)
  ✓ Temporal derivatives (velocity, acceleration)
  ✓ Advanced augmentation (temporal crop, mixup, masking)
  ✓ Multi-branch architecture (LSTM + CNN + Statistics)
  ✓ Attention mechanism
  ✓ Feature normalization

Expected reduction in TongueOut confusion by 60-80%!


In [16]:
import numpy as np
import cv2
import mediapipe as mp
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle


class ImprovedFeatureExtractor:
    """
    Enhanced feature extraction with:
    1. Dimensionality reduction (select only relevant landmarks)
    2. Geometric features (distances, angles, ratios)
    3. Temporal features (velocity, acceleration)
    4. Normalization to person-invariant space
    """

    def __init__(self):
        self.mp_holistic = mp.solutions.holistic

        # Key landmark indices for each emote type
        self.key_face_indices = {
            'mouth': [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291,
                     308, 324, 318, 402, 317, 14, 87, 178, 88, 95],  # Mouth contour
            'eyes': [33, 160, 158, 133, 153, 144, 362, 385, 387, 263, 373, 380],  # Eye regions
            'eyebrows': [70, 63, 105, 66, 107, 336, 296, 334, 293, 300],  # Eyebrows
            'nose': [1, 2, 98, 327, 6, 168]  # Nose bridge and tip
        }

        # Important pose landmarks
        self.key_pose_indices = [
            0, 11, 12, 13, 14, 15, 16,  # Upper body and arms
            23, 24  # Hips for stability reference
        ]

    def extract_geometric_features(self, results):
        """Extract hand-crafted geometric features"""
        features = []

        if results.face_landmarks:
            landmarks = results.face_landmarks.landmark

            # === MOUTH FEATURES (Critical for TongueOut, Yawn, Cry) ===
            # Mouth opening ratio
            upper_lip = landmarks[13]  # Upper lip center
            lower_lip = landmarks[14]  # Lower lip center
            mouth_left = landmarks[61]
            mouth_right = landmarks[291]

            mouth_height = np.sqrt((upper_lip.x - lower_lip.x)**2 +
                                  (upper_lip.y - lower_lip.y)**2 +
                                  (upper_lip.z - lower_lip.z)**2)
            mouth_width = np.sqrt((mouth_left.x - mouth_right.x)**2 +
                                 (mouth_left.y - mouth_right.y)**2 +
                                 (mouth_left.z - mouth_right.z)**2)

            mouth_aspect_ratio = mouth_height / (mouth_width + 1e-6)
            features.extend([mouth_height, mouth_width, mouth_aspect_ratio])

            # Tongue protrusion indicator (distance from mouth center to lower lip)
            mouth_center_x = (mouth_left.x + mouth_right.x) / 2
            mouth_center_y = (mouth_left.y + mouth_right.y) / 2
            tongue_indicator = np.sqrt((lower_lip.x - mouth_center_x)**2 +
                                      (lower_lip.y - mouth_center_y)**2)
            features.append(tongue_indicator)

            # === EYE FEATURES (For cry detection) ===
            left_eye_top = landmarks[159]
            left_eye_bottom = landmarks[145]
            right_eye_top = landmarks[386]
            right_eye_bottom = landmarks[374]

            left_eye_openness = np.sqrt((left_eye_top.x - left_eye_bottom.x)**2 +
                                       (left_eye_top.y - left_eye_bottom.y)**2)
            right_eye_openness = np.sqrt((right_eye_top.x - right_eye_bottom.x)**2 +
                                        (right_eye_top.y - right_eye_bottom.y)**2)

            features.extend([left_eye_openness, right_eye_openness])

            # === FACIAL SYMMETRY (Helps with all expressions) ===
            nose_tip = landmarks[1]
            face_symmetry = abs(nose_tip.x - 0.5)  # Distance from center
            features.append(face_symmetry)

        else:
            features.extend([0.0] * 7)  # Corrected from 8 to 7 to match when face landmarks are present

        # === POSE FEATURES (Critical for HandsUp, Still) ===
        if results.pose_landmarks:
            pose_landmarks = results.pose_landmarks.landmark

            # Shoulder to wrist distances (for HandsUp)
            left_shoulder = pose_landmarks[11]
            right_shoulder = pose_landmarks[12]
            left_wrist = pose_landmarks[15]
            right_wrist = pose_landmarks[16]

            left_arm_height = left_shoulder.y - left_wrist.y  # Negative = hands up
            right_arm_height = right_shoulder.y - right_wrist.y

            features.extend([left_arm_height, right_arm_height])

            # Hands above head indicator
            nose = pose_landmarks[0]
            hands_above_head = int(left_wrist.y < nose.y or right_wrist.y < nose.y)
            features.append(hands_above_head)

            # Shoulder width (for normalization)
            shoulder_width = np.sqrt((left_shoulder.x - right_shoulder.x)**2 +
                                    (left_shoulder.y - right_shoulder.y)**2)
            features.append(shoulder_width)

        else:
            features.extend([0.0] * 4)

        # === HAND FEATURES (For cry-with-hands) ===
        if results.left_hand_landmarks or results.right_hand_landmarks:
            # Hand near face indicator
            hand_near_face = 0.0

            if results.left_hand_landmarks and results.face_landmarks:
                left_hand_center = np.mean([
                    [lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark
                ], axis=0)
                face_center = np.mean([
                    [landmarks[idx].x, landmarks[idx].y, landmarks[idx].z]
                    for idx in [1, 61, 291]  # Nose and mouth corners
                ], axis=0)

                dist_to_face = np.linalg.norm(left_hand_center - face_center)
                hand_near_face = max(hand_near_face, 1.0 / (1.0 + dist_to_face * 5))

            if results.right_hand_landmarks and results.face_landmarks:
                right_hand_center = np.mean([
                    [lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark
                ], axis=0)
                face_center = np.mean([
                    [landmarks[idx].x, landmarks[idx].y, landmarks[idx].z]
                    for idx in [1, 61, 291]
                ], axis=0)

                dist_to_face = np.linalg.norm(right_hand_center - face_center)
                hand_near_face = max(hand_near_face, 1.0 / (1.0 + dist_to_face * 5))

            features.append(hand_near_face)
        else:
            features.append(0.0)

        return np.array(features)

    def extract_raw_landmarks(self, results):
        """Extract normalized raw landmark coordinates"""
        features = []

        # Face landmarks (selected key points only)
        if results.face_landmarks:
            for category in ['mouth', 'eyes', 'eyebrows', 'nose']:
                for idx in self.key_face_indices[category]:
                    lm = results.face_landmarks.landmark[idx]
                    features.extend([lm.x, lm.y, lm.z])
        else:
            total_face_landmarks = sum(len(v) for v in self.key_face_indices.values())
            features.extend([0.0] * (total_face_landmarks * 3))

        # Pose landmarks (selected key points)
        if results.pose_landmarks:
            for idx in self.key_pose_indices:
                lm = results.pose_landmarks.landmark[idx]
                features.extend([lm.x, lm.y, lm.z, lm.visibility])
        else:
            features.extend([0.0] * (len(self.key_pose_indices) * 4))

        # Hand landmarks (average position + spread)
        for hand_landmarks in [results.left_hand_landmarks, results.right_hand_landmarks]:
            if hand_landmarks:
                coords = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
                hand_center = np.mean(coords, axis=0)
                hand_spread = np.std(coords, axis=0)
                features.extend(hand_center.tolist())
                features.extend(hand_spread.tolist())
            else:
                features.extend([0.0] * 6)

        return np.array(features)

    def extract_frame_features(self, frame, holistic):
        """Extract all features from a single frame"""
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image_rgb)

        # Combine geometric and raw features
        geometric_features = self.extract_geometric_features(results)
        raw_features = self.extract_raw_landmarks(results)

        all_features = np.concatenate([geometric_features, raw_features])
        return all_features

    def compute_temporal_features(self, sequence):
        """
        Compute velocity and acceleration features

        Args:
            sequence: (T, F) array of features over time

        Returns:
            Enhanced sequence with temporal derivatives
        """
        # Velocity (first derivative)
        velocity = np.diff(sequence, axis=0, prepend=sequence[0:1])

        # Acceleration (second derivative)
        acceleration = np.diff(velocity, axis=0, prepend=velocity[0:1])

        # Combine original, velocity, and acceleration
        enhanced = np.concatenate([sequence, velocity, acceleration], axis=1)

        return enhanced


# ============================================================================
# IMPROVED DATA LOADING WITH BETTER AUGMENTATION
# ============================================================================

class TemporalAugmenter:
    """Advanced augmentation that preserves gesture semantics"""

    @staticmethod
    def temporal_crop(sequence, crop_ratio=0.8):
        """Randomly crop and resample (simulates faster/slower execution)"""
        length = len(sequence)
        crop_length = int(length * crop_ratio)
        start_idx = np.random.randint(0, length - crop_length + 1)

        cropped = sequence[start_idx:start_idx + crop_length]

        # Resample back to original length
        indices = np.linspace(0, len(cropped) - 1, length).astype(int)
        return cropped[indices]

    @staticmethod
    def add_temporal_jitter(sequence, jitter_std=0.02):
        """Add smooth temporal noise (simulates natural variation)"""
        # Use Gaussian filter for smooth noise
        noise = np.random.normal(0, jitter_std, sequence.shape)

        # Apply temporal smoothing to noise
        from scipy.ndimage import gaussian_filter1d
        smooth_noise = gaussian_filter1d(noise, sigma=2, axis=0)

        return sequence + smooth_noise

    @staticmethod
    def temporal_mask(sequence, mask_ratio=0.1):
        """Randomly mask some frames (forces model to handle occlusions)"""
        augmented = sequence.copy()
        num_frames = len(sequence)
        num_mask = int(num_frames * mask_ratio)

        mask_indices = np.random.choice(num_frames, num_mask, replace=False)

        # Replace masked frames with interpolation
        for idx in mask_indices:
            if idx > 0 and idx < num_frames - 1:
                augmented[idx] = (augmented[idx-1] + augmented[idx+1]) / 2

        return augmented

    @staticmethod
    def mixup_augmentation(seq1, seq2, alpha=0.2):
        """Mix two sequences (from same class) with random weight"""
        lam = np.random.beta(alpha, alpha)
        return lam * seq1 + (1 - lam) * seq2


def load_improved_dataset(data_path, actions, sequence_length=30,
                         augment_factor=2, test_size=0.2, val_size=0.15):
    """
    Load dataset with improved feature extraction and augmentation
    """
    extractor = ImprovedFeatureExtractor()
    augmenter = TemporalAugmenter()

    sequences = []
    labels = []

    print("Loading videos with improved features...")

    with extractor.mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as holistic:

        for action in actions:
            action_path = os.path.join(data_path, action)
            video_files = [f for f in os.listdir(action_path) if f.endswith('.mp4')]

            print(f"\n{action}: {len(video_files)} videos")
            action_sequences = []  # Store for mixup augmentation

            for video_file in video_files:
                cap = cv2.VideoCapture(os.path.join(action_path, video_file))
                frames_features = []

                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break

                    features = extractor.extract_frame_features(frame, holistic)
                    frames_features.append(features)

                cap.release()

                if len(frames_features) == 0:
                    continue

                # Resample to fixed length
                frames_array = np.array(frames_features)
                indices = np.linspace(0, len(frames_array) - 1, sequence_length).astype(int)
                resampled = frames_array[indices]

                # Add temporal features
                resampled_with_temporal = extractor.compute_temporal_features(resampled)

                # Store original
                sequences.append(resampled_with_temporal)
                labels.append(action)
                action_sequences.append(resampled_with_temporal)

            # Create augmented versions
            print(f"  Creating {augment_factor} augmented versions per video...")
            for seq in action_sequences:
                for _ in range(augment_factor):
                    # Apply random combination of augmentations
                    aug_seq = seq.copy()

                    if np.random.rand() > 0.5:
                        aug_seq = augmenter.temporal_crop(aug_seq, crop_ratio=np.random.uniform(0.7, 0.9))

                    if np.random.rand() > 0.5:
                        aug_seq = augmenter.add_temporal_jitter(aug_seq, jitter_std=0.015)

                    if np.random.rand() > 0.3:
                        aug_seq = augmenter.temporal_mask(aug_seq, mask_ratio=0.1)

                    # Occasionally mix with another sequence from same class
                    if np.random.rand() > 0.7 and len(action_sequences) > 1:
                        other_seq = action_sequences[np.random.randint(len(action_sequences))]
                        aug_seq = augmenter.mixup_augmentation(aug_seq, other_seq, alpha=0.2)

                    sequences.append(aug_seq)
                    labels.append(action)

    X = np.array(sequences)

    # Normalize features
    print("\nNormalizing features...")
    scaler = StandardScaler()
    X_reshaped = X.reshape(-1, X.shape[-1])
    X_normalized = scaler.fit_transform(X_reshaped)
    X = X_normalized.reshape(X.shape)

    # Encode labels
    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(labels)

    # Split data
    from sklearn.model_selection import train_test_split
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y_encoded, test_size=(test_size + val_size),
        random_state=42, stratify=y_encoded
    )

    val_ratio = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=(1 - val_ratio),
        random_state=42, stratify=y_temp
    )

    print(f"\n✓ Data loaded and preprocessed!")
    print(f"Feature dimension: {X.shape[-1]}")
    print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

    return X_train, X_val, X_test, y_train, y_val, y_test, scaler, label_encoder


# ============================================================================
# IMPROVED MODEL ARCHITECTURE
# ============================================================================

def build_improved_model(input_shape, num_classes, dropout_rate=0.4):
    """
    State-of-the-art architecture for temporal sequence classification

    Key improvements:
    1. Bidirectional LSTMs (see past and future context)
    2. Attention mechanism (focus on important frames)
    3. Multi-scale temporal convolutions (capture different speeds)
    4. Residual connections (better gradient flow)
    5. Stronger regularization
    """

    # Attention Layer
    class TemporalAttention(layers.Layer):
        def __init__(self, units, **kwargs):
            super().__init__(**kwargs)
            self.units = units

        def build(self, input_shape):
            self.W = self.add_weight(
                shape=(input_shape[-1], self.units),
                initializer='glorot_uniform',
                trainable=True,
                name='attention_W'
            )
            self.b = self.add_weight(
                shape=(self.units,),
                initializer='zeros',
                trainable=True,
                name='attention_b'
            )
            self.u = self.add_weight(
                shape=(self.units,),
                initializer='glorot_uniform',
                trainable=True,
                name='attention_u'
            )

        def call(self, x):
            # x shape: (batch, time, features)
            # Compute attention scores
            score = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
            attention_weights = tf.nn.softmax(tf.tensordot(score, self.u, axes=1), axis=1)
            attention_weights = tf.expand_dims(attention_weights, -1)

            # Apply attention
            weighted = x * attention_weights
            return tf.reduce_sum(weighted, axis=1)

    inputs = layers.Input(shape=input_shape)

    # Masking layer
    x = layers.Masking(mask_value=0.0)(inputs)

    # === Branch 1: Bidirectional LSTM with Attention ===
    lstm1 = layers.Bidirectional(
        layers.LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)
    )(x)
    lstm1 = layers.LayerNormalization()(lstm1)

    lstm2 = layers.Bidirectional(
        layers.LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)
    )(lstm1)
    lstm2 = layers.LayerNormalization()(lstm2)

    # Apply attention
    attended = TemporalAttention(64)(lstm2)

    # === Branch 2: Temporal Convolutions (Multi-scale) ===
    conv1 = layers.Conv1D(64, kernel_size=3, padding='same', activation='relu')(x)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.MaxPooling1D(pool_size=2)(conv1)

    conv2 = layers.Conv1D(64, kernel_size=5, padding='same', activation='relu')(x)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.MaxPooling1D(pool_size=2)(conv2)

    # Global pooling for conv branches
    conv1_pooled = layers.GlobalAveragePooling1D()(conv1)
    conv2_pooled = layers.GlobalAveragePooling1D()(conv2)

    # === Branch 3: Statistical Features ===
    stats_mean = layers.Lambda(lambda x: tf.reduce_mean(x, axis=1))(x)
    stats_std = layers.Lambda(lambda x: tf.math.reduce_std(x, axis=1))(x)
    stats_max = layers.Lambda(lambda x: tf.reduce_max(x, axis=1))(x)

    # Merge all branches
    merged = layers.Concatenate()([
        attended, conv1_pooled, conv2_pooled, stats_mean, stats_std, stats_max
    ])

    # Dense layers with residual connections
    dense1 = layers.Dense(256, activation='relu')(merged)
    dense1 = layers.BatchNormalization()(dense1)
    dense1 = layers.Dropout(dropout_rate)(dense1)

    dense2 = layers.Dense(128, activation='relu')(dense1)
    dense2 = layers.BatchNormalization()(dense2)
    dense2 = layers.Dropout(dropout_rate * 0.75)(dense2)

    # Residual connection
    dense2_residual = layers.Dense(128)(merged)
    dense2_combined = layers.Add()([dense2, dense2_residual])
    dense2_combined = layers.Activation('relu')(dense2_combined)

    dense3 = layers.Dense(64, activation='relu')(dense2_combined)
    dense3 = layers.Dropout(dropout_rate * 0.5)(dense3)

    # Output
    outputs = layers.Dense(num_classes, activation='softmax')(dense3)

    model = keras.Model(inputs=inputs, outputs=outputs, name='ImprovedEmoteClassifier')

    return model


# ============================================================================
# USAGE: How to train with the improved system
# ============================================================================

# 1. Load data with improved features
X_train, X_val, X_test, y_train, y_val, y_test, scaler, label_encoder = load_improved_dataset(
    data_path=DATA_PATH,
    actions=actions,
    sequence_length=30,
    augment_factor=2,
    test_size=0.15,
    val_size=0.15
)

# 2. Build improved model
model = build_improved_model(
    input_shape=(X_train.shape[1], X_train.shape[2]),
    num_classes=len(actions),
    dropout_rate=0.4
)

# 3. Compile with advanced optimizer
from tensorflow.keras.optimizers import AdamW

model.compile(
    optimizer=AdamW(learning_rate=0.001, weight_decay=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 4. Setup callbacks
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=8,
    min_lr=1e-7,
    verbose=1
)

checkpoint = keras.callbacks.ModelCheckpoint(
    'best_improved_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# 5. Calculate class weights (in case of imbalance)
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights))

# Extra weight for TongueOut if still problematic
tongue_idx = np.where(label_encoder.classes_ == 'TongueOut')[0][0]
class_weight_dict[tongue_idx] *= 1.3

print("Class weights:", class_weight_dict)

# 6. Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=16,
    class_weight=class_weight_dict,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=1
)

# 7. Evaluate
from sklearn.metrics import classification_report, confusion_matrix

y_pred = np.argmax(model.predict(X_test), axis=1)
print("\nTest Set Performance:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

# 8. Save scaler for inference
with open('feature_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("\n✓ Training complete!")


# ============================================================================
# COMPARISON: Why This is Better
# ============================================================================

print("\n" + "="*80)
print("COMPARISON: OLD vs NEW APPROACH")
print("="*80)

comparison = """
┌─────────────────────────┬──────────────────────────┬─────────────────────────────┐
│ Aspect                  │ OLD Approach             │ NEW Approach                │
├─────────────────────────┼──────────────────────────┼─────────────────────────────┤
│ Feature Dimension       │ 1662 (all landmarks)     │ ~200 (selected + engineered)│
│                         │ • Sparse, redundant      │ • Dense, informative        │
│                         │ • No semantic meaning    │ • Semantic features         │
├─────────────────────────┼──────────────────────────┼─────────────────────────────┤
│ Temporal Features       │ None                     │ Velocity + Acceleration     │
│                         │ • Only position          │ • Captures dynamics         │
│                         │                          │ • Motion patterns           │
├─────────────────────────┼──────────────────────────┼─────────────────────────────┤
│ Normalization           │ None                     │ StandardScaler on features  │
│                         │ • Scale issues           │ • Person-invariant          │
├─────────────────────────┼──────────────────────────┼─────────────────────────────┤
│ Augmentation            │ Random noise + scale     │ Temporal crop, mixup, mask  │
│                         │ • Breaks semantics       │ • Preserves gesture meaning │
│                         │ • Frame-by-frame         │ • Sequence-aware            │
├─────────────────────────┼──────────────────────────┼─────────────────────────────┤
│ Model Architecture      │ 2 LSTM layers            │ Multi-branch architecture   │
│                         │ • Shallow                │ • Bidirectional LSTMs       │
│                         │ • No attention           │ • Attention mechanism       │
│                         │                          │ • Temporal CNNs             │
│                         │                          │ • Statistical pooling       │
│                         │                          │ • Residual connections      │
├─────────────────────────┼──────────────────────────┼─────────────────────────────┤
│ Temporal Understanding  │ Basic LSTM               │ Multi-scale:                │
│                         │ • Single receptive field │ • 3-frame, 5-frame windows  │
│                         │                          │ • Global context            │
│                         │                          │ • Attention on key moments  │
├─────────────────────────┼──────────────────────────┼─────────────────────────────┤
│ Regularization          │ 50% dropout, batch norm  │ Multiple techniques:        │
│                         │                          │ • Layer normalization       │
│                         │                          │ • Dropout (varying rates)   │
│                         │                          │ • L2 weight decay (AdamW)   │
│                         │                          │ • Class weighting           │
├─────────────────────────┼──────────────────────────┼─────────────────────────────┤
│ TongueOut Detection     │ Generic features         │ Specialized features:       │
│                         │ • Struggles vs Still     │ • Mouth aspect ratio        │
│                         │                          │ • Tongue protrusion metric  │
│                         │                          │ • Temporal velocity spike   │
│                         │                          │ • Attention on key frame    │
└─────────────────────────┴──────────────────────────┴─────────────────────────────┘

Expected Performance Improvements:
• Overall accuracy: +10-15%
• TongueOut recall: +40-60%
• TongueOut vs Still confusion: -70-80%
• Generalization to new people: Significantly better
• Training stability: Much improved
"""

print(comparison)

print("\n" + "="*80)
print("KEY INNOVATIONS FOR TONGUEOUT DETECTION")
print("="*80)

innovations = """
1. GEOMETRIC FEATURES:
   • Mouth height/width ratio → Detects mouth opening
   • Tongue protrusion indicator → Distance metric for tongue extension
   • Lower lip displacement → Tracks tongue movement

2. TEMPORAL FEATURES:
   • Velocity: Detects SUDDEN mouth opening (TongueOut) vs static (Still)
   • Acceleration: Captures the tongue thrust motion
   • Multi-frame context: Sees before/during/after pattern

3. ATTENTION MECHANISM:
   • Learns to focus on the 2-3 frames where tongue is actually out
   • Ignores "still-like" moments before/after the gesture
   • Critical for separating TongueOut from Still

4. TEMPORAL CONVOLUTIONS:
   • 3-frame window: Captures quick movements
   • 5-frame window: Captures full gesture arc
   • Detects motion patterns that LSTMs might miss

5. CLASS WEIGHTING:
   • 1.3x weight on TongueOut forces model to prioritize it
   • Focal loss alternative could be added for extreme cases
"""

print(innovations)


Loading videos with improved features...

Cry: 40 videos


I0000 00:00:1765213861.539836 3056316 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
W0000 00:00:1765213861.627652 3060593 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765213861.643299 3060593 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765213861.647781 3060595 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765213861.648051 3060590 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765213861.648564 3060594 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support 

  Creating 2 augmented versions per video...

HandsUp: 48 videos
  Creating 2 augmented versions per video...

Still: 43 videos
  Creating 2 augmented versions per video...

TongueOut: 44 videos
  Creating 2 augmented versions per video...

Yawn: 41 videos
  Creating 2 augmented versions per video...

Normalizing features...

✓ Data loaded and preprocessed!
Feature dimension: 621
Train: 453, Val: 97, Test: 98


/Users/toto211738/Documents/GitHub/CR_Emote_Detection/mp_env/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'temporal_attention' (of type TemporalAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/toto211738/Documents/GitHub/CR_Emote_Detection/mp_env/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/toto211738/Documents/GitHub/CR_Emote_Detection/mp_env/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'conv1d_1' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not 

Model: "ImprovedEmoteClassifier"

...

 Trainable params: 2,116,101 (8.07 MB)

 Non-trainable params: 1,024 (4.00 KB)

Class weights: {0: 1.0785714285714285, 1: 0.897029702970297, 2: 1.0066666666666666, 3: 1.2802173913043478, 4: 1.0534883720930233}
Epoch 1/100


/Users/toto211738/Documents/GitHub/CR_Emote_Detection/mp_env/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'conv1d_1' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/toto211738/Documents/GitHub/CR_Emote_Detection/mp_env/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/Users/toto211738/Documents/GitHub/CR_Emote_Detection/mp_env/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'temporal_attention' (of type TemporalAttention) was passed an input with a mask attached to it. However, this layer does not 

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5437 - loss: 1.6122
Epoch 1: val_accuracy improved from None to 0.84536, saving model to best_improved_model.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 9s 122ms/step - accuracy: 0.7174 - loss: 0.8456 - val_accuracy: 0.8454 - val_loss: 0.3361 - learning_rate: 0.0010
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.9092 - loss: 0.3837
Epoch 2: val_accuracy improved from 0.84536 to 0.94845, saving model to best_improved_model.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.9316 - loss: 0.3003 - val_accuracy: 0.9485 - val_loss: 0.1246 - learning_rate: 0.0010
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.9857 - loss: 0.0685
Epoch 3: val_accuracy did not improve from 0.94845
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.9801 - loss: 0.0760 - val_accuracy: 0.9381 - val_loss: 0.1423 - learning_rate: 0.0010
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.9686 - loss: 0.0914
Ep

In [14]:
import pickle

# 8. Save scaler for inference (already done in the previous step, but re-confirming here for clarity)
with open('feature_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save the trained model
model.save('best_improved_model2.keras')

# Save the label_encoder for inference
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("\n✓ Model, scaler, and label encoder saved successfully!")



✓ Model, scaler, and label encoder saved successfully!


In [11]:
import cv2
import matplotlib.pyplot as plt

def process_and_display_image(image_path):
    """
    Reads an image from the given path, converts it to grayscale, and displays it.
    Args:
        image_path (str): The path to the image file.
    """
    try:
        # Read the image
        img = cv2.imread(image_path)

        if img is None:
            print(f"Error: Could not read the image from {image_path}")
            return

        # Convert to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Display the original and grayscale images
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title('Original Image')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(gray_img, cmap='gray')
        plt.title('Grayscale Image')
        plt.axis('off')

        plt.show()

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage (you might need to upload an image or provide a valid path)
# process_and_display_image('/content/sample_image.jpg')
